In [1]:
get_ipython().system('pip install transformers')
get_ipython().system('pip install torch')
get_ipython().system('pip install -r requirements.txt')

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("GanjinZero/UMLSBert_ENG")
model = AutoModel.from_pretrained("GanjinZero/UMLSBert_ENG")
import pandas as pd

In [3]:
# read in scraped mayo symptoms
with open("mayo_data.json", 'r') as j:
     data = json.loads(j.read())

FileNotFoundError: [Errno 2] No such file or directory: 'mayo_data.json'

In [4]:
symptoms=[]
specialties=[]
diseases=[]

In [5]:
for disease in list(data.keys()):
    list_of_symptoms=data[disease]['symptoms']
    if len(list_of_symptoms)==0:
        pass
    symptoms.append(list_of_symptoms)
    diseases.append([disease]*len(list_of_symptoms))
    list_of_specialties=data[disease]['specialties']
    specialtystr=','.join(list_of_specialties)
    specialties.append([specialtystr]*len(list_of_symptoms))

NameError: name 'data' is not defined

In [6]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
diseases=flatten(diseases)
symptoms=flatten(symptoms)
specialties=flatten(specialties)

In [8]:
lst = [diseases,symptoms,specialties]
df = pd.DataFrame(
    {'disease': diseases,
     'symptoms': symptoms,
     'specialties': specialties
    })

In [9]:
def get_pooled_embedding(text, model, tokenizer):
    tokenized_input = tokenizer(text, return_tensors='pt')
    output = model(**tokenized_input)
    
    return output.pooler_output.detach().numpy()[0]

In [10]:
def create_pooled_embedding_df(model, tokenizer, df):
    df['pooled_embedding'] = df.symptoms.apply(get_pooled_embedding, args=(model, tokenizer))
    
    return df

In [11]:
# add embedding column and save to pickle obj
df= create_pooled_embedding_df(model, tokenizer, df)
print(df)
#df.to_pickle("mayo_symptoms_embeddings.plk")

Empty DataFrame
Columns: [disease, symptoms, specialties, pooled_embedding]
Index: []
